In [1]:
# Load packages
from util.loggers import createLogHandler
from input_handler.data_preprocessor import data_preprocessor
from input_handler.load_parameters import load_configuration_file, load_parameters
from input_handler.env_setting import run_setting
from pathlib import Path
import pandas as pd
import numpy as np
import datetime
from scipy.stats import norm

from scipy.stats import norm
from typing import Tuple, Dict


import warnings
warnings.filterwarnings("ignore")


# show all columns and rows of a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
configPath = Path(r'C:\Users\SA814XM\Engagement\03_KhanBank\kb_ecl_engine\khb_engine\run_config_file.json')
c = load_configuration_file(configPath=configPath)
rc = run_setting(run_config=c)

In [3]:
logger = createLogHandler('ecl_calc', rc.logPath/'Log_file_risk_parameter_generation.log')
logger.info('********** Initiate ECL Engine Calculator **********')

dp = data_preprocessor(context=rc)

# Load parameters
logger.info('Loading all necessary parameters ...')
print('------- Loading parameter files -------') 

try:
        param = load_parameters(parmPath=rc.parmPath)
except Exception as e:
        logger.exception("message")
else:
        logger.info('Parameter loading complete.')

# load scenario data
logger.info('Loading scenario data ...')
print('------- Loading scenario data -------') 

try:
        # npl_data_path = Path("C:\\Users\\SA814XM\\OneDrive - EY\\99_Data_Server\\scenario\\2024Q2\\data")
        npl_data = dp.load_scenario_data(
            data_path=rc.dataPathScen, file_pattern='NPL')
except Exception as e:
        logger.exception("message")
else:
        logger.info('Scenario data loading complete.')


# Load input data
logger.info('Reading ECL input files ...')
print('------- Reading ECL input files -------') 

try:
        instr_df, _, _ = dp.load_input_data(param=param)

        _, _, sa_df, _, _ = dp.run(instr_df=instr_df,
                                   run_scope=rc.run_scope,
                                   param=param)
        
        fs_df, is_error = dp.standardize_data(param_dict=param,
                                      inDataPath=rc.inDataPath,
                                      rawDataName=rc.sa_fs_table_name,
                                      inputDataExt=rc.inputDataExtECL,
                                      dtype_tbl=rc.dtype_tbl,
                                      )

        other_df, is_error = dp.standardize_data(param_dict=param,
                                        inDataPath=rc.inDataPath,
                                        rawDataName=rc.sa_other_debt_table_name,
                                        inputDataExt=rc.inputDataExtECL,
                                        dtype_tbl=rc.dtype_tbl,
                                        )

        repay_df, is_error = dp.standardize_data(param_dict=param,
                                                inDataPath=rc.inDataPath,
                                                rawDataName=rc.repayment_table_name,
                                                inputDataExt=rc.inputDataExtECL,
                                                dtype_tbl=rc.dtype_tbl)
        
except Exception as e:
        logger.exception("message")
else:
        logger.info('Input data read successful.')


------- Loading parameter files -------
------- Loading scenario data -------
------- Reading ECL input files -------


In [4]:
# get param dfs
es_df = param['Economic_sector']
pwa_df = param['pwa']

period = param['CFL_LGD_period'].Period.values[0]

In [5]:
# get class parmas
base_year = rc.prev_yymm
scenario_version = rc.SCENARIO_VERSION

In [6]:
# get lists used
custs = fs_df['CUST_ID'].unique().tolist()

### Cash flow projection

In [8]:
######## Cash flow projection (FS profit) ########

# Get AutoFSParam - growth rate for working capital
# TODO the starting point of the for loop: for cust in custs:
cust = custs[0]
print(cust)
sa_df_cust = sa_df[sa_df['CUST_ID'] == cust].copy()


DNINULZAV


In [9]:
def get_adjustment_factor(sa_df, other_df):
    # Get adjustment factor = bal in sa_table / (bal in other_debt + bal in sa_table)
    other_df_bal = other_df[other_df['CUST_ID'] == cust].TOTAL_DRAWN_BAL_LCY.sum()
    sa_df_bal = sa_df_cust[sa_df_cust['CUST_ID'] == cust].DRAWN_BAL_LCY.sum()
    adj_factor = sa_df_bal / (sa_df_bal + other_df_bal)
    
    return adj_factor

adj_factor = get_adjustment_factor(sa_df_cust, other_df)
adj_factor

1.0

In [10]:
from datetime import date

def generate_fy_keys(base_year: int, lookback: int = 2, lookforward: int = 3) -> dict:
    """Generate financial year keys with clear temporal relationships."""
    fy_dict = {}
    
    # Previous years
    for i in range(1, lookback + 1):
        fy_dict[f'prev_{i}'] = date(base_year - i, 12, 31)
    
    # Current year
    fy_dict['curr'] = date(base_year, 12, 31)
    
    # Future years
    for i in range(1, lookforward + 1):
        fy_dict[f'next_{i}'] = date(base_year + i, 12, 31)
    
    return fy_dict

# Usage
fy_dict = generate_fy_keys(int(str(rc.prev_yymm)[:4]))
fy_dict

{'prev_1': datetime.date(2022, 12, 31),
 'prev_2': datetime.date(2021, 12, 31),
 'curr': datetime.date(2023, 12, 31),
 'next_1': datetime.date(2024, 12, 31),
 'next_2': datetime.date(2025, 12, 31),
 'next_3': datetime.date(2026, 12, 31)}

In [11]:

# TODO: incorporate the ITEM_NAME chk in pre_run_validation
fs_df_ = fs_df[fs_df['CUST_ID']==cust].copy()
fs_df_['ITEM_NAME'] = fs_df_['ITEM_NAME'].str.strip().str.replace(' ', '_')

# Ensure STATEMENT_DATE is in date format (not datetime)
fs_df_['STATEMENT_DATE'] = pd.to_datetime(fs_df_['STATEMENT_DATE']).dt.date

# Get unique ITEM_NAME lists for BS and IS
wc_list = fs_df_[fs_df_['STATEMENT_TYPE'] == 'BS']['ITEM_NAME'].unique().tolist()
cash_list = fs_df_[fs_df_['STATEMENT_TYPE'] == 'IS']['ITEM_NAME'].unique().tolist()

# Pivot the table
fs_df_1 = fs_df_.pivot_table(
    index=['ITEM_NAME'], 
    columns='STATEMENT_DATE', 
    values='ITEM_AMOUNT', 
    aggfunc='sum'
).reset_index()

# Separate balance sheet - working capital items
wc_df = fs_df_1[fs_df_1['ITEM_NAME'].isin(wc_list)]

# Separate income statement - cash xxx
cash_df = fs_df_1[fs_df_1['ITEM_NAME'].isin(cash_list)]


In [12]:
fs_df

,CUST_ID,STATEMENT_DATE,STATEMENT_TYPE,STATEMENT_SECTION,ITEM_ID,ITEM_NAME,ITEM_AMOUNT,ITEM_CCY_CD
0,DNINULZAV,2021-12-31,BS,CA,1,Accounts/trade receivables,1.482700e+10,MNT
1,DNINULZAV,2021-12-31,BS,CA,2,Advance to suppliers,8.638000e+09,MNT
2,DNINULZAV,2021-12-31,BS,CA,3,Intercompany receivables,0.000000e+00,MNT
3,DNINULZAV,2021-12-31,BS,CA,4,Inventory,3.908500e+10,MNT
4,DNINULZAV,2021-12-31,BS,CA,5,Other receivables,2.319000e+09,MNT
5,DNINULZAV,2021-12-31,BS,CA,6,Other short term assets,0.000000e+00,MNT
6,DNINULZAV,2021-12-31,BS,CL,1,Accounts payables,3.106300e+10,MNT
7,DNINULZAV,2021-12-31,BS,CL,2,Advances from customer,1.718500e+10,MNT
8,DNINULZAV,2021-12-31,BS,CL,3,Intercompany payables,0.000000e+00,MNT
9,DNINULZAV,2021-12-31,BS,CL,4,Other current liabilities,6.143800e+10,MNT


In [13]:
wc_df

STATEMENT_DATE,ITEM_NAME,2021-12-31,2022-12-31,2023-12-31
0,Accounts/trade_receivables,1.482700e+10,3.004020e+10,5.611976e+10
1,Accounts_payables,3.106300e+10,5.361070e+10,5.045326e+10
2,Accumulated_depreciation,0.000000e+00,0.000000e+00,0.000000e+00
3,Advance_to_suppliers,8.638000e+09,1.185700e+10,2.699876e+10
4,Advances_from_customer,1.718500e+10,5.540360e+10,7.408466e+10
7,Intercompany_payables,0.000000e+00,0.000000e+00,0.000000e+00
8,Intercompany_receivables,0.000000e+00,0.000000e+00,0.000000e+00
10,Inventory,3.908500e+10,4.137400e+10,4.946260e+10
12,Other_current_liabilities,6.143800e+10,2.908580e+10,4.472488e+10
15,Other_receivables,2.319000e+09,2.080000e+09,3.151680e+09


In [14]:
cash_df

STATEMENT_DATE,ITEM_NAME,2021-12-31,2022-12-31,2023-12-31
5,COGS,2.174620e+11,3.933110e+11,6.815122e+11
6,Depreciation_expense,1.046400e+10,9.661000e+09,0.000000e+00
9,Interest_expense,9.845000e+09,9.177000e+09,2.873443e+10
11,Operating_expense,6.550000e+09,2.001700e+10,3.477737e+10
13,Other_expense,2.498000e+09,9.107000e+09,1.286483e+10
14,Other_income,1.991000e+10,6.659000e+09,1.867605e+10
17,Realized_FX_gain_(loss),9.490000e+08,-4.930000e+09,-8.920850e+09
18,Sales,2.145050e+11,4.830370e+11,8.143116e+11
20,Taxes,5.500000e+07,5.281000e+09,1.426338e+10


In [15]:
def get_cash_profit(cash_df, fy_dict):
    """Calculate cash-based net profit using rolling window averages and clear temporal references."""
    # Calculate rolling averages using explicit temporal labels
    cash_df[fy_dict['next_1']] = cash_df[[fy_dict['prev_2'], fy_dict['prev_1'], fy_dict['curr']]].mean(axis=1)
    cash_df[fy_dict['next_2']] = cash_df[[fy_dict['prev_1'], fy_dict['curr'], fy_dict['next_1']]].mean(axis=1)
    cash_df[fy_dict['next_3']] = cash_df[[fy_dict['curr'], fy_dict['next_1'], fy_dict['next_2']]].mean(axis=1)
    
    # Initialize results dataframe with clear structure
    cash_profit_df = pd.DataFrame({
        'ITEM_NAME': ['Net_profit', 'Depreciation']
    })
    
    # Unified calculation for each forecast year
    for year_offset in (1, 2, 3):
        year_key = f'next_{year_offset}'
        
        # Explicit column reference for readability
        data = {
            'Sales': cash_df.loc[cash_df['ITEM_NAME'] == 'Sales', fy_dict[year_key]].values[0],
            'COGS': cash_df.loc[cash_df['ITEM_NAME'] == 'COGS', fy_dict[year_key]].values[0],
            'Operating_expense': cash_df.loc[cash_df['ITEM_NAME'] == 'Operating_expense', fy_dict[year_key]].values[0],
            'Depreciation_expense': cash_df.loc[cash_df['ITEM_NAME'] == 'Depreciation_expense', fy_dict[year_key]].values[0],
            'Other_income': cash_df.loc[cash_df['ITEM_NAME'] == 'Other_income', fy_dict[year_key]].values[0],
            'Other_expense': cash_df.loc[cash_df['ITEM_NAME'] == 'Other_expense', fy_dict[year_key]].values[0],
            'Realized_FX_gain_(loss)': cash_df.loc[cash_df['ITEM_NAME'] == 'Realized_FX_gain_(loss)', fy_dict[year_key]].values[0],
            'Interest_expense': cash_df.loc[cash_df['ITEM_NAME'] == 'Interest_expense', fy_dict[year_key]].values[0],
            'Taxes': cash_df.loc[cash_df['ITEM_NAME'] == 'Taxes', fy_dict[year_key]].values[0]
        }
        
        # Clear financial calculations
        ebit = (
            data['Sales'] 
            - data['COGS'] 
            - data['Operating_expense'] 
            - data['Depreciation_expense'] 
            + data['Other_income'] 
            - data['Other_expense'] 
            + data['Realized_FX_gain_(loss)']
        )
        net_profit = ebit - data['Interest_expense'] - data['Taxes']
        
        # Structured assignment
        cash_profit_df.loc[
            cash_profit_df['ITEM_NAME'] == 'Net_profit', 
            fy_dict[year_key]
        ] = net_profit
        
        cash_profit_df.loc[
            cash_profit_df['ITEM_NAME'] == 'Depreciation', 
            fy_dict[year_key]
        ] = data['Depreciation_expense']
    
    return cash_profit_df

# Usage with clear temporal labels
cash_profit_df = get_cash_profit(cash_df=cash_df, fy_dict=fy_dict)
cash_profit_df

,ITEM_NAME,2024-12-31,2025-12-31,2026-12-31
0,Net_profit,2.620553e+10,3.877737e+10,3.896583e+10
1,Depreciation,6.708333e+09,5.456444e+09,4.054926e+09


In [489]:
# Generate FSParam

# code_mapping = {
#         'Accounts_payables': 'APR',
#         'Accounts/trade_receivables': 'ARR',
#         'Accumulated_depreciation': 'ADR',
#         'Advance_to_suppliers': 'ASR',
#         'Advances_from_customer': 'ACR',
#         'Intercompany_payables': 'IPX',
#         'Intercompany_receivables': 'IRX',
#         'Inventory': 'INR',
#         'Other_current_liabilities': 'CLX',
#         'Other_receivables': 'ORX',
#         'Other_short_term_assets': 'SAX',
#         'Tax_payable': 'TPR'
#     }

In [18]:
def get_growth_rate(df, numerator_col, denominator_col):
    """Calculate growth rate with robust error handling"""
    result = df[numerator_col] / df[denominator_col] - 1
    return result.replace([np.inf, -np.inf, np.nan], 0)

def get_changes_in_working_capital(wc_df: pd.DataFrame, fy_dict: dict) -> pd.DataFrame:
    """
    Calculate projected changes in working capital using historical growth patterns.
    Applies negative sign convention for asset increases.
    
    Parameters:
        wc_df (pd.DataFrame): Working capital dataframe containing historical values
        fy_dict (dict): Fiscal year mapping with temporal keys ('prev_*', 'curr', 'next_*')
        
    Returns:
        pd.DataFrame: Contains projected working capital changes for future periods
    """
    
    # List of asset items requiring negative sign convention
    ASSET_ITEMS = [
        'Accounts/trade_receivables',
        'Other_receivables',
        'Intercompany_receivables',
        'Inventory',
        'Advance_to_suppliers',
        'Other_short_term_assets'
    ]

    df = wc_df.copy()

    # 1. Calculate historical growth rates
    df['gr_prev_1'] = get_growth_rate(df, fy_dict['prev_1'], fy_dict['prev_2'])
    df['gr_curr'] = get_growth_rate(df, fy_dict['curr'], fy_dict['prev_1'])

    # 2. Project growth rates
    df['gr_next_1'] = df[['gr_prev_1', 'gr_curr']].mean(axis=1)
    df['gr_next_2'] = df[['gr_curr', 'gr_next_1']].mean(axis=1)
    df['gr_next_3'] = df[['gr_next_1', 'gr_next_2']].mean(axis=1)
    
    df_gr = df.copy()
    
    # 3. Calculate projected balances
    df['tmp_1'] = df[fy_dict['curr']] * (1 + df['gr_next_1'])
    df['tmp_2'] = df['tmp_1'] * (1 + df['gr_next_2'])
    df['tmp_3'] = df['tmp_2'] * (1 + df['gr_next_3'])

    # 4. Calculate changes
    df[fy_dict['next_1']] = df['tmp_1'] - df[fy_dict['curr']]
    df[fy_dict['next_2']] = df['tmp_2'] - df['tmp_1']
    df[fy_dict['next_3']] = df['tmp_3'] - df['tmp_2']

    # 5. Apply negative sign convention for assets
    asset_mask = df['ITEM_NAME'].isin(ASSET_ITEMS)
    for year in ['next_1', 'next_2', 'next_3']:
        df.loc[asset_mask, fy_dict[year]] *= -1

    return df[['ITEM_NAME', fy_dict['next_1'], fy_dict['next_2'], fy_dict['next_3']]], df_gr

ciwc_df = get_changes_in_working_capital(wc_df, fy_dict)[0]
ciwc_df

STATEMENT_DATE,ITEM_NAME,2024-12-31,2025-12-31,2026-12-31
0,Accounts/trade_receivables,-5.315108e+10,-9.917729e+10,-1.933074e+11
1,Accounts_payables,1.682551e+10,9.237085e+09,1.801118e+10
2,Accumulated_depreciation,0.000000e+00,0.000000e+00,0.000000e+00
3,Advance_to_suppliers,-2.226975e+10,-5.177812e+10,-9.477067e+10
4,Advances_from_customer,9.487031e+10,1.366632e+11,3.192855e+11
7,Intercompany_payables,0.000000e+00,0.000000e+00,0.000000e+00
8,Intercompany_receivables,-0.000000e+00,-0.000000e+00,-0.000000e+00
10,Inventory,-6.283339e+09,-8.989917e+09,-9.331608e+09
12,Other_current_liabilities,2.483352e+08,1.221563e+10,7.925595e+09
15,Other_receivables,-6.495126e+08,-1.370929e+09,-1.465630e+09


In [19]:
fs_param_df = get_changes_in_working_capital(wc_df, fy_dict)[1]
fs_param_df

STATEMENT_DATE,ITEM_NAME,2021-12-31,2022-12-31,2023-12-31,gr_prev_1,gr_curr,gr_next_1,gr_next_2,gr_next_3
0,Accounts/trade_receivables,1.482700e+10,3.004020e+10,5.611976e+10,1.026047,0.868155,0.947101,0.907628,0.927365
1,Accounts_payables,3.106300e+10,5.361070e+10,5.045326e+10,0.725870,-0.058896,0.333487,0.137296,0.235391
2,Accumulated_depreciation,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Advance_to_suppliers,8.638000e+09,1.185700e+10,2.699876e+10,0.372656,1.277031,0.824843,1.050937,0.937890
4,Advances_from_customer,1.718500e+10,5.540360e+10,7.408466e+10,2.223951,0.337181,1.280566,0.808874,1.044720
7,Intercompany_payables,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Intercompany_receivables,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
10,Inventory,3.908500e+10,4.137400e+10,4.946260e+10,0.058565,0.195500,0.127032,0.161266,0.144149
12,Other_current_liabilities,6.143800e+10,2.908580e+10,4.472488e+10,-0.526583,0.537688,0.005553,0.271620,0.138586
15,Other_receivables,2.319000e+09,2.080000e+09,3.151680e+09,-0.103062,0.515231,0.206085,0.360658,0.283371


In [491]:

# def get_auto_fs_param(fy_dict, wc_df, cash_profit_df):

#     wc_param_table = pd.melt(wc_df, id_vars=['ITEM_NAME'], var_name='Variable_name', value_name='Multiplier', value_vars=['gr_by_n1', 'gr_by_n2', 'gr_by_n3'])
#     wc_param_table.sort_values(by=['ITEM_NAME', 'Variable_name'], inplace=True)
    
#     c_param_table = pd.melt(cash_profit_df, id_vars=['ITEM_NAME'], var_name='Variable_name', value_name='Multiplier', value_vars=[fy_dict['by_n1'], fy_dict['by_n2'], fy_dict['by_n3']])
#     c_param_table.sort_values(by=['ITEM_NAME', 'Variable_name'], inplace=True)
    
#     fs_param_table = pd.concat([wc_param_table, c_param_table], ignore_index=True)
#     fs_param_table['Multiplier'] = 1.00 # defualt multiplier value is 1.00, means the growth rate is unadjusted
    
#     # create Projection_code
#     statement_types = []
#     projection_years = []
#     descriptions = []

#     for index, row in fs_param_table.iterrows():
        
#         if row['ITEM_NAME'] in wc_list:
#             for year in (1,2,3):
#                 if row['Variable_name'] == f'gr_by_n{year}':
#                     statement_types.append('BS')
                    
#                     projection_year = fy_dict[f'by_n{year}'].year
#                     projection_years.append(projection_year)
                    
#                     description = f'Multiplier applied to the projected growth rate of {row["ITEM_NAME"]} in year {year}'
#                     descriptions.append(description) 
                
#         elif row['ITEM_NAME'] in ['Net_profit', "Depreciation"]:
#             for year in (1,2,3):
#                 if row['Variable_name'] == fy_dict[f'by_n{year}']:
#                     statement_types.append('IS')
                    
#                     projection_year = fy_dict[f'by_n{year}'].year
#                     projection_years.append(projection_year)
                    
#                     description = f'Multiplier applied to the projected amount of {row["ITEM_NAME"]} in year {year}'
#                     descriptions.append(description) 
            
#         else:
#             statement_types.append(None)
#             projection_years.append(None)
#             descriptions.append(None)
            
#     fs_param_table['Statement_type'] = statement_types
#     fs_param_table['Projection_year'] = projection_years
#     fs_param_table['Description'] = descriptions
#     fs_param_table['Cust_ID'] = cust
#     fs_param_table['Index'] = range(1, len(fs_param_table) + 1)

#     # TODO confirm if the Projection_code is category type or str type
#     header = {
#                 "Index": ["Parameter name", "Parameter description", "Mandatory input", "Parameter data type"],
#                 "Cust_ID": ["Cust_ID", "customer ID", "Y", "str"],
#                 "ITEM_NAME": ["Parameter_name", "Parameter description", "Y", "str"],
#                 "Statement_type": ["Statement_type", "Statement type", "Y", "str"],
#                 "Projection_year": ["Projection_year", "Projection_year", "Y", "int"],
#                 "Description": ["Description", "The description of projection", "N", "str"],
#                 "Multiplier": ["Multiplier", "The projected growth rate", "Y", "float"]
#             }

#     table_header = pd.DataFrame(header)
#     table_content = fs_param_table[['Index', 'Cust_ID', 'ITEM_NAME', 'Statement_type', 'Projection_year', 'Description', 'Multiplier']].copy()
#     fs_param = pd.concat( [table_header, table_content], ignore_index=True)
    
#     return fs_param

# fs_param = get_auto_fs_param(cash_profit_df=cash_profit_df, wc_df=wc_df, fy_dict=fy_dict)

In [492]:
def get_sa_params_df(cash_profit_df: pd.DataFrame, ciwc_df: pd.DataFrame, fy_dict: dict) -> pd.DataFrame:
    """
    Calculate strategic analysis parameters for cash flow and working capital changes.
    
    Parameters:
        cash_profit_df (pd.DataFrame): DataFrame containing Net_profit and Depreciation
        ciwc_df (pd.DataFrame): DataFrame with working capital changes
        fy_dict (dict): Fiscal year mapping with 'next_1', 'next_2', 'next_3' keys
        
    Returns:
        pd.DataFrame: Parameters DataFrame with cash and ciwc values for each forecast period
    """
    
    # Validate fiscal year keys
    required_keys = ['next_1', 'next_2', 'next_3']
    if not all(k in fy_dict for k in required_keys):
        missing = [k for k in required_keys if k not in fy_dict]
        raise ValueError(f"Missing fiscal year keys: {missing}")

    # Component definitions
    WC_COMPONENTS = {
        'negative': [
            'Accounts/trade_receivables',
            'Intercompany_receivables',
            'Other_receivables',
            'Inventory',
            'Advance_to_suppliers',
            'Other_short_term_assets'
        ],
        'positive': [
            'Accounts_payables',
            'Intercompany_payables',
            'Tax_payable',
            'Advances_from_customer',
            'Other_current_liabilities'
        ]
    }

    # 1. Calculate cash parameters (Net Profit + Depreciation)
    cash_values = []
    for year_key in required_keys:
        year_col = fy_dict[year_key]
        try:
            np = cash_profit_df.loc[cash_profit_df['ITEM_NAME'] == 'Net_profit', year_col].values[0]
            dp = cash_profit_df.loc[cash_profit_df['ITEM_NAME'] == 'Depreciation', year_col].values[0]
            cash_values.append(np + dp)
        except IndexError:
            raise ValueError(f"Missing Net_profit or Depreciation in {year_key}")

    # 2. Calculate working capital changes
    ciwc_values = []
    for year_key in required_keys:
        year_col = fy_dict[year_key]
        try:
            negative = ciwc_df.loc[ciwc_df['ITEM_NAME'].isin(WC_COMPONENTS['negative']), year_col].sum()
            positive = ciwc_df.loc[ciwc_df['ITEM_NAME'].isin(WC_COMPONENTS['positive']), year_col].sum()
            ciwc_values.append(positive + negative)
        except KeyError:
            raise ValueError(f"Missing working capital components in {year_col}")

    # 3. Construct final DataFrame
    return pd.DataFrame({
        'Parameter': ['cash', 'ciwc'],
        fy_dict['next_1']: [cash_values[0], ciwc_values[0]],
        fy_dict['next_2']: [cash_values[1], ciwc_values[1]],
        fy_dict['next_3']: [cash_values[2], ciwc_values[2]]
    })

sa_params_df = get_sa_params_df(cash_profit_df, ciwc_df, fy_dict)
sa_params_df

,Parameter,2024-12-31,2025-12-31,2026-12-31
0,cash,3.291386e+10,4.423381e+10,4.302075e+10
1,ciwc,-3.097999e+10,-2.992405e+11,-1.016467e+12


In [493]:
# TODO: update the logic multiplier

multiplier_df = pd.DataFrame({
    'Parameter': ['cash', 'ciwc'],
    fy_dict['next_1']: [1, 1],
    fy_dict['next_2']: [1, 1],
    fy_dict['next_3']: [1, 0.1]
})


def apply_multipliers(sa_params_df, multiplier_df):
    # Merge the two DataFrames on the 'Parameter' column
    merged_df = sa_params_df.merge(multiplier_df, on='Parameter', suffixes=('_sa', '_multiplier'))
    
    # Multiply the corresponding columns
    for year_key in ['next_1', 'next_2', 'next_3']:
        year_col = fy_dict[year_key]
        merged_df[year_col] = merged_df[f"{year_col}_sa"] * merged_df[f"{year_col}_multiplier"]

    # Drop the intermediate columns
    merged_df = merged_df.drop(columns=[f"{col}_sa" for col in [fy_dict['next_1'], fy_dict['next_2'], fy_dict['next_3']]] + 
                              [f"{col}_multiplier" for col in [fy_dict['next_1'], fy_dict['next_2'], fy_dict['next_3']]])
    
    return merged_df

sa_params_df_adj = apply_multipliers(sa_params_df, multiplier_df)
sa_params_df_adj


,Parameter,2024-12-31,2025-12-31,2026-12-31
0,cash,3.291386e+10,4.423381e+10,4.302075e+10
1,ciwc,-3.097999e+10,-2.992405e+11,-1.016467e+11


In [494]:
def get_net_operating_cashflow(sa_params_df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates all parameter rows into a 'noc' (Net Operating Cash) row
    
    Parameters:
        sa_params_df: DataFrame with columns ['Parameter', 'next_1', 'next_2', 'next_3']
        
    Returns:
        DataFrame with original rows plus new 'noc' row containing column sums
        
    Example Input:
        | Parameter | next_1 | next_2 | next_3 |
        |-----------|--------|--------|--------|
        | cash      | 150    | 165    | 182    |
        | ciwc      | -50    | -55    | -60    |
        
    Example Output:
        | Parameter | next_1 | next_2 | next_3 |
        |-----------|--------|--------|--------|

        | noc       | 100    | 110    | 122    |
    """
    # Create a copy to avoid modifying original data
    df = sa_params_df.copy()
    
    # Calculate sums for each forecast period
    noc_values = {
        'Parameter': 'noc',
        fy_dict['next_1']: df[fy_dict['next_1']].sum(),
        fy_dict['next_2']: df[fy_dict['next_2']].sum(),
        fy_dict['next_3']: df[fy_dict['next_3']].sum()
    }
    
    # Append new row using concat for pandas>=1.4.0 compatibility
    noc_row = pd.DataFrame([noc_values])
    return noc_row

noc_df = get_net_operating_cashflow(sa_params_df=sa_params_df_adj)
noc_df

,Parameter,2024-12-31,2025-12-31,2026-12-31
0,noc,1.933874e+09,-2.550067e+11,-5.862590e+10


In [495]:
# Export noc_df for all custs


### NPL statistics

In [496]:
def _calculate_npl_stats(data: pd.Series, alpha: float) -> Tuple[float, float, float]:
    """Helper function to calculate NPL statistics"""
    n = len(data)
    mean = data.mean()
    std = data.std()
    
    z_score = norm.ppf(1 - alpha/2)
    std_err = std / (n ** 0.5)

    return (
        mean,
        mean - z_score * std_err,
        mean + z_score * std_err
    )

def calculate_sector_npl_stats(
    es_df: pd.DataFrame,
    npl_data: pd.DataFrame,
    alpha: float = 0.1
) -> pd.DataFrame:
    """
    Calculate NPL statistics with confidence intervals for economic sectors.
    
    Args:
        es_df: DataFrame containing economic sector information with 'ECON_SECTOR_TYPE' column
        npl_data: DataFrame containing NPL data with columns named 'SECTOR_NPL_{sector}'
        alpha: Significance level for confidence intervals (default: 0.1)
    
    Returns:
        DataFrame with columns ['sector', 'mean', 'ci_lower', 'ci_upper']
    """
    # Validate inputs
    if 'ECON_SECTOR_TYPE' not in es_df:
        raise ValueError("es_df must contain 'ECON_SECTOR_TYPE' column")
    
    if not 0 < alpha < 1:
        raise ValueError("Alpha must be between 0 and 1")

    # Generate sector names
    sectors = es_df['ECON_SECTOR_TYPE'].unique().tolist()
    npl_columns = [f'SECTOR_NPL_{sec}' for sec in sectors]
    
    # Verify all required columns exist in npl_data
    missing_cols = [col for col in npl_columns if col not in npl_data]
    if missing_cols:
        raise ValueError(f"Missing NPL columns in npl_data: {missing_cols}")

    # Calculate statistics for each sector
    stats = []
    for sector, col in zip(sectors, npl_columns):
        data = npl_data[col].dropna()
        if len(data) < 2:
            raise ValueError(f"Insufficient data for sector {sector}")
            
        mean, ci_lower, ci_upper = _calculate_npl_stats(data, alpha)
        stats.append({
            'npl_sector': sector,
            'mean': mean,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper
        })

    return pd.DataFrame(stats)


npl_stats = calculate_sector_npl_stats(es_df, npl_data, alpha=0.1)
npl_stats

,npl_sector,mean,ci_lower,ci_upper
0,P,0.191967,0.168333,0.215602
1,O,0.116156,0.097645,0.134667


### LGD

In [497]:
repay_df.head()

,REPORT_DATE,XTRT_DATE,CF_DATE,CONTRACT_ID,UNPAID_PRIN_BAL_BEG_OCY,PRIN_PMT_OCY,INT_PMT_OCY
0,2024-06-30,2024-06-30,2025-01-31,4435,1.378649e+07,7.535071e+05,0.0
1,2024-06-30,2024-06-30,2025-11-30,4552,5.162833e+07,6.058156e+05,0.0
2,2024-06-30,2024-06-30,2025-10-31,4592,1.676733e+07,2.161544e+05,0.0
3,2024-06-30,2024-06-30,2026-04-30,4599,1.249553e+08,1.700269e+06,0.0
4,2024-06-30,2024-06-30,2029-01-31,4599,6.593702e+07,1.874899e+06,0.0


In [498]:
def get_total_repay_df(df, repay_df):
    contract_ids = df.CONTRACT_ID.unique().tolist()
    cust_repay = repay_df[repay_df['CONTRACT_ID'].isin(contract_ids)]
    
    ttl_repay = cust_repay.pivot_table(index='CF_DATE', columns='CONTRACT_ID', values='PRIN_PMT_OCY')
    ttl_repay['ttl_repay'] = ttl_repay.sum(axis=1)
    
    return ttl_repay[['ttl_repay']]

ttl_repay_df = get_total_repay_df(sa_df_cust, repay_df)
ttl_repay_df

CONTRACT_ID,ttl_repay
CF_DATE,
2024-07-31,2.149005e+09
2024-08-31,2.152991e+09
2024-09-30,2.158027e+09
2024-10-31,2.163621e+09
2024-11-30,2.168262e+09
2024-12-31,2.135187e+09
2025-01-31,1.666668e+09
2025-02-28,1.666668e+09
2025-03-31,1.666668e+09


In [499]:
from typing import Dict, Optional
import pandas as pd
from pandas.tseries.offsets import MonthEnd

def get_cash_flow_dates(scenario_date: str, period_years: int) -> pd.DatetimeIndex:
    """
    Generate monthly cash flow dates starting from the quarter following the scenario date.
    
    Args:
        scenario_date: Format 'YYYYQX' where X is quarter (1-4)
        period_years: Number of years to project forward
        
    Returns:
        Monthly DatetimeIndex through end of projection period
        
    Example:
        >>> get_cash_flow_dates('2023Q4', 5)
        DatetimeIndex(['2024-01-31', '2024-02-29', ..., '2027-12-31'], dtype='datetime64[ns]', freq='M')
    """
    # Validate input format
    if len(scenario_date) != 6 or not scenario_date[4] == 'Q':
        raise ValueError("Scenario date must be in format 'YYYYQX' (e.g. '2023Q4')")
    
    year = int(scenario_date[:4])
    quarter = int(scenario_date[-1])
    
    if not 1 <= quarter <= 4:
        raise ValueError("Quarter must be between 1-4")

    # Calculate start date
    if quarter == 4:
        start_year = year + 1
        start_month = 1
    else:
        start_year = year
        start_month = quarter * 3 + 1  # Next quarter start month
        
    start_date = pd.Timestamp(year=start_year, month=start_month, day=1) + MonthEnd(1)
    
    # Calculate end date
    end_date = pd.Timestamp(year + period_years - 1, 12, 31)
    
    return pd.date_range(start=start_date, end=end_date, freq='M')

cash_flow_dates = get_cash_flow_dates(rc.SCENARIO_VERSION, period)
cash_flow_dates

DatetimeIndex(['2024-07-31', '2024-08-31', '2024-09-30', '2024-10-31',
               '2024-11-30', '2024-12-31', '2025-01-31', '2025-02-28',
               '2025-03-31', '2025-04-30', '2025-05-31', '2025-06-30',
               '2025-07-31', '2025-08-31', '2025-09-30', '2025-10-31',
               '2025-11-30', '2025-12-31', '2026-01-31', '2026-02-28',
               '2026-03-31', '2026-04-30', '2026-05-31', '2026-06-30',
               '2026-07-31', '2026-08-31', '2026-09-30', '2026-10-31',
               '2026-11-30', '2026-12-31', '2027-01-31', '2027-02-28',
               '2027-03-31', '2027-04-30', '2027-05-31', '2027-06-30',
               '2027-07-31', '2027-08-31', '2027-09-30', '2027-10-31',
               '2027-11-30', '2027-12-31', '2028-01-31', '2028-02-29',
               '2028-03-31', '2028-04-30', '2028-05-31', '2028-06-30',
               '2028-07-31', '2028-08-31', '2028-09-30', '2028-10-31',
               '2028-11-30', '2028-12-31'],
              dtype='datetime64[n

In [500]:

def get_pwa_noc(
    fy_dict: Dict[str, int],
    sa_df_cust: pd.DataFrame,
    noc_df: pd.DataFrame,
    npl_stats: pd.DataFrame,
    pwa_df: pd.DataFrame,
    cust: str
) -> Optional[pd.DataFrame]:
    """
    Calculate Probability-Weighted Average Net Operating Cash (NOC) for a customer.
    
    Args:
        fy_dict: Fiscal year mapping {'next_1': 2024, ...}
        sa_df_cust: Strategic analysis dataframe with customer-sector mapping
        noc_df: NOC projections dataframe
        npl_stats: Sector NPL statistics with confidence intervals
        pwa_df: Scenario probability weights
        cust_id: Target customer ID
        
    Returns:
        DataFrame with PWA NOC projections or None if invalid input
    """
        
    sectors = sa_df_cust['ECON_SECTOR'].unique()
    if len(sectors) > 1:
        print(f"Customer {cust} has multiple sectors: {sectors}")
        return None
        
    sector_type = es_df.loc[es_df['ECON_SECTOR'] == sectors[0], 'ECON_SECTOR_TYPE'].iloc[0]
    
    # Get NPL risk factor
    try:
        sector_stats = npl_stats.query(f"npl_sector == '{sector_type}'").iloc[0]
        risk_factor = (sector_stats['mean'] - sector_stats['ci_lower']) / sector_stats['mean']

    except IndexError:
        print(f"No NPL stats found for sector {sector_type}")
        return None

    # Calculate scenario-adjusted NOC
    results = []
    for year_key in ['next_1', 'next_2', 'next_3']:
        if year_key not in fy_dict:
            print(f"Missing fiscal year key {year_key}")
            return None
            
        fiscal_year = fy_dict[year_key]
        base_noc = noc_df.get(fiscal_year, 0)
        
        # Scenario adjustments
        scenarios = {
            'SEVE': base_noc * (1 - risk_factor),
            'BASE': base_noc,
            'GROW': base_noc * (1 + risk_factor)
        }
        
        # Apply probability weights
        pwa_noc = sum(
            scenarios[scenario] * pwa_df.loc[pwa_df['scenario'] == scenario, 'pwa'].values[0]
            for scenario in ['SEVE', 'BASE', 'GROW']
        )

        results.append({'prediction_year': fiscal_year, 'pwa_noc': pwa_noc.iloc[0]
})
    
    return pd.DataFrame(results)

# TODO: change the noc_df back after the params are decided
noc_df_fake = pd.DataFrame({
    'Parameter': ['noc'],
    fy_dict['next_1']: [150033428750.2],
    fy_dict['next_2']: [13410669664.88],
    fy_dict['next_3']: [67990473450.6879]
})

pwa_noc_df = get_pwa_noc(fy_dict, sa_df_cust, noc_df, npl_stats, pwa_df, cust)
pwa_noc_df

,prediction_year,pwa_noc
0,2024-12-31,1.943357e+09
1,2025-12-31,-2.562571e+11
2,2026-12-31,-5.891338e+10


In [501]:
def get_total_exposure(df):
    return df['PRIN_BAL_LCY'].sum() + df['ACRU_INT_LCY'].sum()

ttl_exposure = get_total_exposure(sa_df_cust)
ttl_exposure

23043575847.559998

In [502]:
def get_cust_EIR(df):
    eir = df['EFF_INT_RT']
    exposure = df['PRIN_BAL_LCY'] + df['ACRU_INT_LCY']

    return (eir * exposure).sum() / exposure.sum()

eir = get_cust_EIR(sa_df_cust)
eir

0.09401258930872665

In [503]:
def get_lgd_df(cash_flow_dates: pd.DatetimeIndex, pwa_noc_df: pd.DataFrame) -> pd.DataFrame:
    """
    Create Loss Given Date (LGD) DataFrame with datetime index preservation.
    
    Args:
        cash_flow_dates: Monthly cash flow dates (DatetimeIndex)
        pwa_noc_df: Annual NOC projections with prediction years
        
    Returns:
        DataFrame with cash flow dates as index and total NOC values
    """
    # Convert prediction years to datetime and extract year
    annual_noc = pwa_noc_df.copy()
    annual_noc['year'] = pd.to_datetime(annual_noc['prediction_year']).dt.year

    # Create base dataframe with cash flow dates as index
    lgd_df = pd.DataFrame(index=cash_flow_dates)
    lgd_df.index.name = 'cash_flow_date'
    
    # Extract year from index and prepare for merge
    lgd_df = lgd_df.assign(year=lgd_df.index.year).reset_index()
    
    # Merge with annual NOC values
    lgd_df = lgd_df.merge(
        annual_noc[['year', 'pwa_noc']],
        on='year',
        how='left'
    )
    
    # Restore datetime index and clean up
    lgd_df = lgd_df.set_index('cash_flow_date').sort_index()
    lgd_df = lgd_df[['pwa_noc']].rename(columns={'pwa_noc': 'ttl_noc'})
    
    # Forward fill within each year group and fill remaining NAs
    lgd_df['ttl_noc'] = lgd_df['ttl_noc'].ffill().fillna(0)
    
    return lgd_df


# Usage
lgd_df = get_lgd_df(cash_flow_dates, pwa_noc_df)
lgd_df

,ttl_noc
cash_flow_date,
2024-07-31,1.943357e+09
2024-08-31,1.943357e+09
2024-09-30,1.943357e+09
2024-10-31,1.943357e+09
2024-11-30,1.943357e+09
2024-12-31,1.943357e+09
2025-01-31,-2.562571e+11
2025-02-28,-2.562571e+11
2025-03-31,-2.562571e+11


In [504]:
# left joing ttl_repay_df to lgd_df
lgd_df_1 = lgd_df.merge(ttl_repay_df, how='left', left_index=True, right_index=True).copy()
lgd_df_1 = lgd_df_1.fillna(0)
lgd_df_1

,ttl_noc,ttl_repay
cash_flow_date,,
2024-07-31,1.943357e+09,2.149005e+09
2024-08-31,1.943357e+09,2.152991e+09
2024-09-30,1.943357e+09,2.158027e+09
2024-10-31,1.943357e+09,2.163621e+09
2024-11-30,1.943357e+09,2.168262e+09
2024-12-31,1.943357e+09,2.135187e+09
2025-01-31,-2.562571e+11,1.666668e+09
2025-02-28,-2.562571e+11,1.666668e+09
2025-03-31,-2.562571e+11,1.666668e+09


In [505]:
def add_discounted_col(
    df: pd.DataFrame,
    lgd_df: pd.DataFrame,
    col: str,
    eir: float
) -> pd.DataFrame:
    """
    Add discounted NOC column using time-aware discounting.
    
    Args:
        lgd_df: DataFrame with cash_flow_date index and ttl_noc column
        reporting_date: Base date for discounting (e.g. '2024-06-30')
        eir: Effective interest rate (e.g. 0.05 for 5%)
        
    Returns:
        Modified DataFrame with discounted_noc column
    """
    # Convert reporting date to pandas timestamp if not already
    # TODO: chk if the report_date of one cust is unique and = the scenario date
    reporting_date = df['REPORT_DATE'].unique()[0]
    
    # Calculate time delta in years
    delta_days = (lgd_df.index - reporting_date).days
    delta_years = delta_days / 365
    
    # Calculate discount factor
    discount_factor = (1 + eir) ** delta_years
    
    # Calculate discounted NOC
    lgd_df[f'discounted_{col}'] = lgd_df[col] / discount_factor
    
    return lgd_df

# Usage
lgd_df_2 = add_discounted_col(
    sa_df_cust,
    lgd_df_1,
    'ttl_noc',
    eir=eir
)

lgd_df_2 = add_discounted_col(
    sa_df_cust,
    lgd_df_1,
    'ttl_repay',
    eir=eir
)

lgd_df_2


,ttl_noc,ttl_repay,discounted_ttl_noc,discounted_ttl_repay
cash_flow_date,,,,
2024-07-31,1.943357e+09,2.149005e+09,1.928583e+09,2.132667e+09
2024-08-31,1.943357e+09,2.152991e+09,1.913921e+09,2.120380e+09
2024-09-30,1.943357e+09,2.158027e+09,1.899839e+09,2.109702e+09
2024-10-31,1.943357e+09,2.163621e+09,1.885396e+09,2.099091e+09
2024-11-30,1.943357e+09,2.168262e+09,1.871523e+09,2.088115e+09
2024-12-31,1.943357e+09,2.135187e+09,1.857295e+09,2.040630e+09
2025-01-31,-2.562571e+11,1.666668e+09,-2.430470e+11,1.580751e+09
2025-02-28,-2.562571e+11,1.666668e+09,-2.413775e+11,1.569892e+09
2025-03-31,-2.562571e+11,1.666668e+09,-2.395424e+11,1.557958e+09


In [506]:
# def calculate_total_recovery(lgd_df_1):
#     """
#     计算总可回收金额，逻辑如下：
#     1. 每月用discounted_noc偿还ttl_repay，若不足则未还部分累积到下个月。
#     2. 若discounted_noc <= 0，则当月无法还款，未还部分累积到下个月。
#     3. 累积的可用现金流(carry_over)和未偿还债务(remaining_debt)动态传递到后续月份。
#     """
#     total_recovery = 0.0
#     carry_over = 0.0      # 累积的可用现金流
#     remaining_debt = 0.0  # 未偿还的债务
    
#     # 按时间顺序处理数据
#     df = lgd_df_1.sort_index()
    
#     for date, row in df.iterrows():
#         discounted_noc = row["discounted_ttl_noc"]
#         ttl_repay = row["discounted_ttl_repay"]
        
#         # 如果当月discounted_noc <= 0，无法还款
#         current_noc = max(discounted_noc, 0.0)
#         available_cash = carry_over + current_noc
        
#         # 总需还款 = 当月应还 + 上月未还债务
#         total_repay_required = ttl_repay + remaining_debt
        
#         if available_cash >= total_repay_required:
#             # 足够还款：全额偿还，剩余现金流累积到下个月
#             total_recovery += total_repay_required
#             carry_over = available_cash - total_repay_required
#             remaining_debt = 0.0
#         else:
#             # 不足还款：全部可用金额用于还款，未还部分累积到下个月
#             total_recovery += available_cash
#             remaining_debt = total_repay_required - available_cash
#             carry_over = 0.0  # 不足时不可累积
        
#     return total_recovery

# ttl_recover = calculate_total_recovery(lgd_df_1)
# ttl_recover

In [507]:
def get_total_recovery(lgd_df_1):
    """
    Calculate the total recoverable amount (delayed repayment plan version)
    New logic: If the cash flow is insufficient in a given month, the entire repayment plan is shifted back by one month until the cumulative cash flow is sufficient to cover the repayment.
    """
    total_recovery = 0.0
    carry_over = 0.0      # Cumulative available cash flow
    pending_repayments = []  # Queue for pending repayment plans
    
    # Process data in chronological order
    df = lgd_df_1.sort_index()
    
    for date, row in df.iterrows():
        discounted_noc = row["discounted_ttl_noc"]
        ttl_repay = row["discounted_ttl_repay"]
        
        # Add the current month's repayment plan to the queue
        pending_repayments.append(ttl_repay)
        
        # Handle negative cash flow
        current_noc = max(discounted_noc, 0.0)
        carry_over += current_noc  # Accumulate cash flow
        
        # Attempt to process the first repayment plan in the queue
        while len(pending_repayments) > 0 and carry_over >= pending_repayments[0]:
            # Repay the earliest outstanding debt
            repaid = pending_repayments.pop(0)
            total_recovery += repaid
            carry_over -= repaid
    
    return total_recovery

ttl_recover = get_total_recovery(lgd_df_1)
ttl_recover


10549955261.858742

In [508]:
lgd = 1 - ttl_recover/ttl_exposure
lgd

0.5421736916332003